In [87]:
GENE_PERTURBATIONS_DATA_URL = 'http://amp.pharm.mssm.edu/CREEDS/download/single_gene_perturbations-v1.0.json'
DISEASES_DATA_URL = 'http://amp.pharm.mssm.edu/CREEDS/download/disease_signatures-v1.0.json'
DRUG_PERTURBATIONS_DATA_URL = 'http://amp.pharm.mssm.edu/CREEDS/download/single_drug_perturbations-v1.0.json'

In [175]:
from tqdm.notebook import tqdm
import json, requests, os.path


if not os.path.isfile("gene_data.json"):
    response = requests.get(GENE_PERTURBATIONS_DATA_URL, stream=True)
    with open("gene_data.json", "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)
            
if not os.path.isfile("disease_data.json"):
    response = requests.get(DISEASES_DATA_URL, stream=True)
    with open("disease_data.json", "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)
            
if not os.path.isfile("drug_data.json"):
    response = requests.get(DRUG_PERTURBATIONS_DATA_URL, stream=True)
    with open("drug_data.json", "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)

        
gene_data = json.load(open("gene_data.json", "r"))
disease_data = json.load(open("disease_data.json", "r"))
drug_data = json.load(open("drug_data.json", "r"))

In [157]:
def get_data(value, ids, data):
    result = pd.DataFrame(columns=['Disease/Drug', 'Regulation', 'Gene_Name', 'Fold_Change'])
    valid_experiments = [experiment for experiment in data if experiment['id'] in ids]
    for experiment in valid_experiments:
        temp = []
        for gene, fc_value in experiment['down_genes']:
            temp.append([value, 'down', gene, fc_value])
        for gene, fc_value in experiment['up_genes']:
            temp.append([value, 'up', gene, fc_value])
        result = result.append(temp)
    return result

In [171]:
def get_creed_results():
    flag = True
    while(flag):
        CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'

        input_var = str(input('Enter the name of the disease/drug you are searching for. \n'))

        response = requests.get(CREEDS_URL + 'search', params={'q': input_var})
        if response.status_code == 200:
            response_data = response.json()
            ids = [experiment['id'] for experiment in response.json()]

        for data, suffix in zip([gene_data, disease_data, drug_data], ['gene_data', 'disease_data', 'drug_data']):
            result = get_data(value=input_var, ids=ids, data=data)
            if result.empty == True:
                continue
            result.to_csv(
                f'{input_var.lower().replace(" ", "_")}_{suffix}.tsv', 
                sep='\t'
            )
        flag = str(input('Do you want to stop?'))

In [174]:
get_creed_results()

Enter the name of the disease/drug you are searching for. 
Pimozide
Do you want to stop?n
Enter the name of the disease/drug you are searching for. 
Pimozide
Do you want to stop?
